## Ant Colony Optimization Algorithm
#### *Object Replacement Variant*
An object CAN be selected twice in a single partial solution

In [169]:
import numpy as np
from anpy.Ant import Ant
import pandas as pd
from anpy.support import random_index_selection
from anpy.Neighborhood import Neighborhood

pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)
pd.set_option('display.width', 1000)

ITERATIONS = 1000
ANTS_NUMS = 100
KNAPSACK_CAPACITY = 750 # Total Knapsack load capacity
EVA_RATE = 0.5 # Evaporation Rate
ALPHA = 1
BETA = 2
PHEROMONE = 1.0 # (Tau) Amount of pheromones deposited on all objects when a solution has been found

nor = {
    'ALPHA' : ALPHA,
    'BETA' : BETA,
    'PHEROMONE' : PHEROMONE,
    'EVA_RATE' : EVA_RATE
}

best_ant_iteration = pd.DataFrame(columns=
    ['Iteration',
    'Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])
worst_ant_iteration = pd.DataFrame(columns=
    ['Iteration',
    'Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])
best_ant_run = pd.DataFrame(columns=
    ['Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])
worst_ant_run = pd.DataFrame(columns=
    ['Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])

In [170]:
def setup_neighborhood() -> Neighborhood:
    # Create Neighborhood
    N1 = Neighborhood(ANTS_NUMS)
    
    # Set Neighborhood's Weights & Profits
    N1.set_weights("weights.txt")
    N1.set_profits("profits.txt")
    N = N1.generate_neighborhood()
    N1.set_neighborhood(N) 
    return N1  

def get_best_ant_iter(iteration:int, ant:Ant):
    best_ant_iteration.loc[iteration] = [iteration,
        ant.id,                
        ant.s,
        ant.z,
        np.sum(ant.s)]
    
def get_worst_ant_iter(iteration:int, ant:Ant):
    worst_ant_iteration.loc[iteration] = [iteration,
        ant.id,                
        ant.s,
        ant.z,
        np.sum(ant.s)]

In [171]:
def aco():
    for iter in range(1, ITERATIONS+1):
        N = setup_neighborhood()
        i = 0
        while i < len(N.ants):
            random_ant = N.ants[np.random.randint(0,len(N.ants))]
            Vc = KNAPSACK_CAPACITY
            while not random_ant.hasWorked:
                objects_probs = N.get_object_prob()
                object_idx = random_index_selection(N.weights, objects_probs)
                obj_weight = N.weights[object_idx]
                if Vc - obj_weight <= 0: random_ant.hasWorked = True;i += 1;break
                random_ant.add_weight_and_value(obj_weight, N.values[object_idx])
                Vc -= obj_weight
        ants_profits = [ant.z for ant in N.ants]    
        get_best_ant_iter(iter, N.ants[np.argmax(ants_profits)])
        get_worst_ant_iter(iter, N.ants[np.argmin(ants_profits)])

In [172]:
# Get solutions from each cycle
aco()

In [173]:
# Display Iteration's Best Ant
# best_ant_iteration.set_index('Iteration', inplace=True)
# best_ant_iteration

In [174]:
# Display Iteration's Worst Ant
# worst_ant_iteration.set_index('Iteration', inplace=True)
# worst_ant_iteration

In [175]:
# Get Run's Best Ant
best_ant_run.loc[1] = best_ant_iteration.loc[np.argmax(best_ant_iteration['Profit (Z)'])+1]
best_ant_run

,Ant ID,Partial Solution (S),Profit (Z),Capacity (V)
1,3,"[73, 73, 120, 98, 118, 70, 120, 77]",1463,749


In [176]:
# Get Run's Worst Ant
worst_ant_run.loc[1] = worst_ant_iteration.loc[np.argmin(worst_ant_iteration['Profit (Z)'])+1]
worst_ant_run

,Ant ID,Partial Solution (S),Profit (Z),Capacity (V)
1,2,"[87, 80, 80, 110, 118, 82, 77]",1207,634
